In [1]:
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD, Adagrad, RMSprop, Adam
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras import regularizers
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from keras import regularizers
from keras.models import Sequential
from keras.layers import Embedding
from keras.optimizers import SGD, Adagrad, RMSprop, Adam
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\prana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\prana\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\prana\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
stop_words = set(stopwords.words("english"))

def preprocess(text):
    # Remove @username
    text = re.sub(r'@\w+', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove punctuation and special characters
    text = re.sub(r'[^a-zA-z0-9\s]', '', text)
    # Tokenize the words
    words = word_tokenize(text)
    # Remove stop words
    words = [word for word in words if word not in stop_words]
    # Stemming
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in words]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in stemmed_words]
    # Join the words
    text = ' '.join(lemmatized_words)
    return text




data = pd.read_csv("Tweets.csv")
df = data[["text","airline_sentiment"]]
df = df[df.airline_sentiment != "neutral"]
df['text'] = df['text'].apply(preprocess)



max_features = 4000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df['text'].values)

X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X)
Y = np.array(df['airline_sentiment'] == "positive", dtype=int)


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)




In [3]:
def build_classifier(optimizer='adam', dropout_rate=0.5, hidden_units=196):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Embedding(max_features, 128, input_length=X_train.shape[1]))
    model.add(tf.keras.layers.SpatialDropout1D(dropout_rate))
    model.add(tf.keras.layers.LSTM(hidden_units, dropout = 0.3, recurrent_dropout = 0.3 ))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(100, activation = tf.nn.relu))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(2, activation = tf.nn.softmax,kernel_regularizer=regularizers.L1(0.01)))

    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    
    return model

In [4]:
model = KerasClassifier(build_fn=build_classifier)

parameters = {'optimizer': ['RMSprop', 'Adam', 'Adamax', 'Nadam', 'Nesterov','Adagrad'],
             'batch_size': [32, 64],
             'epochs': [5, 8, 11, 14],
             'dropout_rate': [0.5, 0.6, 0.7, 0.8],
             'hidden_units': [32, 64, 128]}

C:\Users\prana\AppData\Local\Temp\ipykernel_4312\909094409.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=build_classifier)


In [5]:
grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5)

In [6]:
grid_search = grid_search.fit(X_train, Y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/5
202/202 [==============================] - 8s 31ms/step - loss: 0.5691 - accuracy: 0.8154
Epoch 2/5
202/202 [==============================] - 6s 31ms/step - loss: 0.3556 - accuracy: 0.8838
Epoch 3/5
202/202 [==============================] - 6s 31ms/step - loss: 0.2910 - accuracy: 0.9067
Epoch 4/5
202/202 [==============================] - 6s 31ms/step - loss: 0.2643 - accuracy: 0.9157
Epoch 5/5
51/51 [==============================] - 1s 10ms/step
Epoch 1/5
202/202 [==============================] - 9s 33ms/step - loss: 0.5726 - accuracy: 0.8188
Epoch 2/5
202/202 [==============================] - 7s 33ms/step - loss: 0.3507 - accuracy: 0.8877
Epoch 3/5
202/202 [==============================] - 7s 33ms/step - loss: 0.2838 - accuracy: 0.9106
Epoch 4/5
202/202 [==============================] - 7s 33ms/step - loss: 0.2549 - accuracy: 0.9163
Epoch 5/5
51/51 [==============================] - 1s 10ms/step
Epoch 1/5
202/202 [==============================] - 9s 33ms/step - loss

51/51 [==============================] - 1s 10ms/step
Epoch 1/5
202/202 [==============================] - 9s 35ms/step - loss: 0.5871 - accuracy: 0.8041
Epoch 2/5
202/202 [==============================] - 7s 35ms/step - loss: 0.3280 - accuracy: 0.8952
Epoch 3/5
202/202 [==============================] - 7s 35ms/step - loss: 0.2495 - accuracy: 0.9259
Epoch 4/5
202/202 [==============================] - 7s 35ms/step - loss: 0.2038 - accuracy: 0.9431
Epoch 5/5
51/51 [==============================] - 1s 10ms/step
Epoch 1/5
202/202 [==============================] - 10s 35ms/step - loss: 0.5686 - accuracy: 0.8185
Epoch 2/5
202/202 [==============================] - 7s 34ms/step - loss: 0.3197 - accuracy: 0.9048
Epoch 3/5
202/202 [==============================] - 7s 35ms/step - loss: 0.2463 - accuracy: 0.9287
Epoch 4/5
202/202 [==============================] - 7s 34ms/step - loss: 0.2040 - accuracy: 0.9412
Epoch 5/5
51/51 [==============================] - 1s 11ms/step
Epoch 1/5
202/202

202/202 [==============================] - 10s 52ms/step - loss: 0.2728 - accuracy: 0.9154
Epoch 4/5
202/202 [==============================] - 10s 52ms/step - loss: 0.2380 - accuracy: 0.9251
Epoch 5/5
51/51 [==============================] - 1s 16ms/step
Epoch 1/5
202/202 [==============================] - 12s 52ms/step - loss: 0.5568 - accuracy: 0.8242
Epoch 2/5
202/202 [==============================] - 11s 52ms/step - loss: 0.2802 - accuracy: 0.9123
Epoch 3/5
202/202 [==============================] - 11s 52ms/step - loss: 0.2157 - accuracy: 0.9386
Epoch 4/5
202/202 [==============================] - 11s 52ms/step - loss: 0.1764 - accuracy: 0.9505
Epoch 5/5
51/51 [==============================] - 1s 15ms/step
Epoch 1/5
202/202 [==============================] - 12s 52ms/step - loss: 0.5740 - accuracy: 0.8131
Epoch 2/5
202/202 [==============================] - 11s 52ms/step - loss: 0.2973 - accuracy: 0.9081
Epoch 3/5
202/202 [==============================] - 11s 52ms/step - loss:

202/202 [==============================] - 13s 51ms/step - loss: 0.5682 - accuracy: 0.8193
Epoch 2/5
202/202 [==============================] - 10s 51ms/step - loss: 0.3096 - accuracy: 0.9042
Epoch 3/5
202/202 [==============================] - 10s 51ms/step - loss: 0.2342 - accuracy: 0.9301
Epoch 4/5
202/202 [==============================] - 10s 52ms/step - loss: 0.1948 - accuracy: 0.9423
Epoch 5/5
51/51 [==============================] - 1s 15ms/step
Epoch 1/5
202/202 [==============================] - 12s 52ms/step - loss: 0.8721 - accuracy: 0.7804
Epoch 2/5
202/202 [==============================] - 10s 52ms/step - loss: 0.8157 - accuracy: 0.7939
Epoch 3/5
202/202 [==============================] - 10s 52ms/step - loss: 0.7798 - accuracy: 0.7939
Epoch 4/5
202/202 [==============================] - 10s 51ms/step - loss: 0.7557 - accuracy: 0.7939
Epoch 5/5
51/51 [==============================] - 3s 63ms/step
Epoch 1/5
202/202 [==============================] - 12s 51ms/step - loss:

202/202 [==============================] - 14s 67ms/step - loss: 0.1683 - accuracy: 0.9530
Epoch 5/5
51/51 [==============================] - 1s 17ms/step
Epoch 1/5
202/202 [==============================] - 16s 69ms/step - loss: 0.5552 - accuracy: 0.8304
Epoch 2/5
202/202 [==============================] - 14s 68ms/step - loss: 0.2753 - accuracy: 0.9134
Epoch 3/5
202/202 [==============================] - 14s 68ms/step - loss: 0.2022 - accuracy: 0.9404
Epoch 4/5
202/202 [==============================] - 14s 68ms/step - loss: 0.1682 - accuracy: 0.9505
Epoch 5/5
51/51 [==============================] - 1s 17ms/step
Epoch 1/5
202/202 [==============================] - 15s 66ms/step - loss: 0.6833 - accuracy: 0.7911
Epoch 2/5
202/202 [==============================] - 13s 66ms/step - loss: 0.4956 - accuracy: 0.7945
Epoch 3/5
202/202 [==============================] - 13s 66ms/step - loss: 0.4157 - accuracy: 0.8070
Epoch 4/5
202/202 [==============================] - 13s 66ms/step - loss:

...

202/202 [==============================] - 11s 55ms/step - loss: 0.1208 - accuracy: 0.9667
Epoch 9/11
202/202 [==============================] - 11s 54ms/step - loss: 0.1041 - accuracy: 0.9723
Epoch 10/11
202/202 [==============================] - 11s 54ms/step - loss: 0.0988 - accuracy: 0.9749
Epoch 11/11
51/51 [==============================] - 1s 17ms/step
Epoch 1/11
202/202 [==============================] - 13s 53ms/step - loss: 0.5765 - accuracy: 0.8124
Epoch 2/11
202/202 [==============================] - 11s 53ms/step - loss: 0.3161 - accuracy: 0.9048
Epoch 3/11
202/202 [==============================] - 11s 53ms/step - loss: 0.2347 - accuracy: 0.9274
Epoch 4/11
202/202 [==============================] - 11s 53ms/step - loss: 0.1917 - accuracy: 0.9432
Epoch 5/11
202/202 [==============================] - 11s 53ms/step - loss: 0.1642 - accuracy: 0.9545
Epoch 6/11
202/202 [==============================] - 11s 54ms/step - loss: 0.1443 - accuracy: 0.9599
Epoch 7/11
202/202 [======

202/202 [==============================] - 6s 30ms/step - loss: 0.7201 - accuracy: 0.7947
Epoch 8/11
202/202 [==============================] - 6s 30ms/step - loss: 0.7125 - accuracy: 0.7947
Epoch 9/11
202/202 [==============================] - 6s 30ms/step - loss: 0.7050 - accuracy: 0.7947
Epoch 10/11
202/202 [==============================] - 6s 30ms/step - loss: 0.7005 - accuracy: 0.7947
Epoch 11/11
51/51 [==============================] - 1s 8ms/step
Epoch 1/11
202/202 [==============================] - 7s 30ms/step - loss: 0.8883 - accuracy: 0.7806
Epoch 2/11
202/202 [==============================] - 6s 30ms/step - loss: 0.8273 - accuracy: 0.7950
Epoch 3/11
202/202 [==============================] - 6s 30ms/step - loss: 0.7930 - accuracy: 0.7950
Epoch 4/11
202/202 [==============================] - 6s 30ms/step - loss: 0.7690 - accuracy: 0.7950
Epoch 5/11
202/202 [==============================] - 6s 30ms/step - loss: 0.7499 - accuracy: 0.7950
Epoch 6/11
202/202 [================

202/202 [==============================] - 14s 70ms/step - loss: 0.1110 - accuracy: 0.9687
Epoch 8/11
202/202 [==============================] - 14s 71ms/step - loss: 0.1005 - accuracy: 0.9726
Epoch 9/11
202/202 [==============================] - 14s 70ms/step - loss: 0.0870 - accuracy: 0.9765
Epoch 10/11
202/202 [==============================] - 14s 70ms/step - loss: 0.0725 - accuracy: 0.9824
Epoch 11/11
51/51 [==============================] - 1s 18ms/step
Epoch 1/11
202/202 [==============================] - 16s 70ms/step - loss: 0.5563 - accuracy: 0.8287
Epoch 2/11
202/202 [==============================] - 14s 70ms/step - loss: 0.2859 - accuracy: 0.9126
Epoch 3/11
202/202 [==============================] - 14s 70ms/step - loss: 0.2137 - accuracy: 0.9370
Epoch 4/11
202/202 [==============================] - 14s 70ms/step - loss: 0.1697 - accuracy: 0.9503
Epoch 5/11
202/202 [==============================] - 14s 71ms/step - loss: 0.1504 - accuracy: 0.9574
Epoch 6/11
202/202 [======

125/202 [=================>............] - ETA: 5s - loss: 0.3193 - accuracy: 0.8960

KeyboardInterrupt: 

In [ ]:
print("Best parameters: ", best_parameters)
print("Best accuracy: ", best_accuracy)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(max_fatures, 128, input_length=X_train.shape[1]),
    tf.keras.layers.SpatialDropout1D(0.5),
    tf.keras.layers.LSTM(196, dropout=0.3, recurrent_dropout=0.3),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.4),

In [ ]:

model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, Y_train, epochs=8, batch_size=64, validation_split=0.2, verbose=2)


In [ ]:

score = model.evaluate(X_test, Y_test, verbose=False)
print("loss = ", score[0])
print("accuracy = ", score[1])